# Simpel lineær regression

Lad os fortsætte med at betragte et datasæt på formen

$$

    (x_1,y_1),(x_2,y_2),\dots,(x_n,y_n).

$$

Hvis der er en sammenhæng mellem de to variabler, vil dette ofte blive tydeligt, når vi plotter dem i et scatterplot. Hvis scatterplottet viser en lineær tendens, kan vi beskrive udviklingen af $y$-observationerne med en lineær funktion af $x$-observationerne. 

Når vi finder en passende lineær model, der beskriver denne sammenhæng, kan vi bruge modellen til at forudsige nye $y$-værdier baseret på nye $x$-værdier. Med andre ord kan vi bruge modellen til at *prædiktere* fremtidige observationer af $y$, når vi kender de tilsvarende værdier af $x$. 

In [2]:
from scipy.stats import norm, uniform
import numpy as np
import matplotlib.pyplot as plt 
from myst_nb import glue
from manim import *

# Set the seed for reproducibility
rng = np.random.default_rng(seed=4)

# Generate dataset
n = 10

x = uniform.rvs(size=n, loc=0, scale=15, random_state=rng)
z = norm.rvs(size=n, random_state=rng)
y = x + z

Manim Community v0.18.0.post0

In [4]:
%%manim -v WARNING -qm --format=mp4 PlotPoints

# Config manim
config.media_embed = True
config.media_width = "100%"

animat_green = "#aecc55"
animat_red = "#cc5241"
animat_yellow = "#d9c750"
animat_blue = "#6a90cc"

n = n
x_data = np.round(x, 1)
z_data = np.round(z, 1)
y_data = np.round(y, 1)

class PlotPoints(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        # 1) Title
        title = Text("Data", color=BLACK).to_edge(UP)
        self.play(Write(title))
        self.wait(0.5)

        # ----------------------------------------------------------------
        # 2) Create TWO tables:
        #    (A) Symbolic table => x_1, y_1 etc.
        #    (B) Numeric table => 1.00, 2.00 etc.
        #
        #    Both have a bold header row in normal font size,
        #    and smaller font for the data rows.
        # ----------------------------------------------------------------

        # (A) Symbolic data
        symbolic_data = [
            [f"x_{{{i+1}}}", f"y_{{{i+1}}}"] for i in range(n)
        ]

        symbolic_table = MathTable(
            symbolic_data,
            col_labels=[MathTex(r"\boldsymbol{x}"), MathTex(r"\boldsymbol{y}")],
            include_outer_lines=False,  # no outer box
            h_buff=1.0,
            v_buff=0.6
        ).scale(0.6)
        symbolic_table.to_edge(LEFT)

        # Thinner lines, all black
        symbolic_table.get_vertical_lines().set_stroke(width=1, color=BLACK)
        symbolic_table.get_horizontal_lines().set_stroke(width=1, color=BLACK)
        symbolic_table.get_entries().set_color(BLACK)

        # Adjust font sizes in the table: header row bigger, data rows smaller
        symbolic_rows = symbolic_table.get_rows()
        # Row 0 => header
        for cell in symbolic_rows[0]:
            cell.set(font_size=36)  # normal size for the bold "x", "y"
        # Rows 1..n => smaller
        for row in symbolic_rows[1:]:
            for cell in row:
                cell.set(font_size=24)

        # (B) Numeric data
        numeric_data = [
            [f"{x_data[i]:.1f}", f"{y_data[i]:.1f}"] for i in range(n)
        ]

        numeric_table = MathTable(
            numeric_data,
            col_labels=[MathTex(r"\boldsymbol{x}"), MathTex(r"\boldsymbol{y}")],
            include_outer_lines=False,
            h_buff=1.0,
            v_buff=0.6
        ).scale(0.6)
        numeric_table.to_edge(LEFT)

        numeric_table.get_vertical_lines().set_stroke(width=1, color=BLACK)
        numeric_table.get_horizontal_lines().set_stroke(width=1, color=BLACK)
        numeric_table.get_entries().set_color(BLACK)

        numeric_rows = numeric_table.get_rows()
        # Row 0 => header
        for cell in numeric_rows[0]:
            cell.set(font_size=36)  # normal size for the bold "x", "y"
        # Rows 1..n => smaller
        for row in numeric_rows[1:]:
            for cell in row:
                cell.set(font_size=24)

        # Show the symbolic table
        self.play(FadeIn(symbolic_table))
        self.wait(1)

        # Transform symbolic table -> numeric table
        self.play(ReplacementTransform(symbolic_table, numeric_table))
        self.wait(1)

        # ----------------------------------------------------------------
        # 3) Axes to the right (unscaled), so x,y labels are in standard positions
        # ----------------------------------------------------------------
        x_max = max(x_data) + 1
        y_max = max(y_data) + 1

        ax = Axes(
            x_range=[0, max(x_data) + 2, 2],
            y_range=[0, max(y_data) + 2, 2],
            axis_config={"include_numbers": True, "stroke_color": BLACK},
        ).scale(0.75)
        ax.x_axis.numbers.set_color(BLACK)
        ax.y_axis.numbers.set_color(BLACK)

        # Place the axes to the right of the table with some spacing
        ax.next_to(numeric_table, RIGHT, buff=1)

        # Standard label positions
        labels = ax.get_axis_labels(
            x_label=MathTex("x", color=BLACK).scale(0.75),
            y_label=MathTex("y", color=BLACK).scale(0.75)  
        )  

        self.play(Create(ax), Write(labels))
        self.wait(1)

        # ----------------------------------------------------------------
        # 4) Transform each numeric row into a Dot on the scatterplot
        # ----------------------------------------------------------------
        table_rows = numeric_table.get_rows()
        # The data rows start at index=1 (index=0 is the header)
        for i in range(1, n+1):
            row_mobj = table_rows[i]  # the entire row
            x_val = float(x_data[i-1])
            y_val = float(y_data[i-1])

            dot = Dot(ax.c2p(x_val, y_val), color=animat_blue).scale(0.75)

            # Optional: highlight the row
            self.play(Indicate(row_mobj, color=animat_blue), run_time=0.5)
            # Transform row -> Dot
            self.play(ReplacementTransform(row_mobj, dot), run_time=0.8)
            self.add(dot)  # keep the dot on screen

        self.wait(6)

En lineær udvikling kan beskrives med en ret linje

$$

    y = ax + b,

$$

men i regression benytter vi en anderledes repræsentation, nemlig

$$

    y = \beta_0 + \beta_1 x,

$$ (eq:simpel-lineaer-regression)

hvor $x$ og $y$ er vores variabler i vores datasæt. Vi kalder $y$-variablen for *responsvariablen* og $x$-variablen for *den forklarende variabel*. Vi kalder $\beta_0$ og $\beta_1$ for modelparametre. Grunden til, at vi bruger repræsentationen {eq}`eq:simpel-lineaer-regression`, skyldes, at den er nemmere at generalisere til en multipel lineær regressionsmodel. Dette kan du læse mere om i {numref}`Kapitel %s<ch-multipel-lineaer-regression>`.

Men hvordan finder vi så en ret linje, der bedst beskriver vores datas udvikling? Vi kan ikke blot vælge to tilfældige punkter og tegne en linje gennem dem, som vi lærte i {numref}`Kapitel %s<ch-analytisk-plangeometri>`, da dette kunne føre til en meget unøjagtig model. Hvilke to punkter skulle vi i så fald vælge? I stedet ønsker vi, at alle observationer i datasættet bidrager til at forme modellen. For at opnå dette anvender vi en metode kendt som mindste kvadraters metode i regression.

## Mindste kvadraters metode

Mindste kvadraters metode handler om at finde den linje, der passer bedst til vores datapunkter, ved at minimere summen af de kvadrerede lodrette afstande mellem linjen og datapunkterne.

Lad os betragte datasættet 

$$

    (x_1,y_1),(x_2,y_2),\dots,(x_n,y_n)

$$

Målet er at finde den rette linje, der bedst beskriver sammenhængen mellem punkterne i dette datasæt. Denne proces kaldes også at *fitte* en ret linje til data. Den vil være givet ved forskriften

$$

    f(x) = \beta_0 + \beta_1 x,

$$ (eq:simpel-lineaer-regression-fx)

hvor vi blot har erstattet $y$ på venstresiden i {eq}`eq:simpel-lineaer-regression` med $f(x)$ for at angive, at modellen forudsiger $y$-værdier baseret på $x$. Modellens forudsagte $y$-værdier er netop $f(x)$. For eksempel vil $y_1$ være den *faktiske observation* for $x_1$, mens $f(x_1)$ vil være den *prædikterede værdi* fra modellen for $x_1$. 

Vi kan nu betragte de lodrette afstande mellem hver faktisk observation og den prædikterede værdi fra modellen. For det første datapunkt er afstanden mellem $y_1$ og den prædikterede værdi $f(x_1)$ givet som

$$

    y_1 - f(x_1),

$$

og for det næste datapunkt har vi 

$$

    y_2 - f(x_2),

$$

og så videre for resten af punkterne. 

In [ ]:
from manim import *

In [ ]:
%%manim -v WARNING -qm --format=mp4 SpecificPoint

config.media_embed = True
config.media_width = "100%"

class SpecificPoint(MovingCameraScene):    
    def construct(self):
        self.camera.background_color = WHITE

        # Set up the axes
        ax = Axes(
            x_range=[0, max(x_data)+2, 2],
            y_range=[0, max(y_data)+2, 2],
            axis_config={"include_numbers": True, "color": BLACK},
        )
        ax.x_axis.numbers.set_color(BLACK)
        ax.y_axis.numbers.set_color(BLACK)
        labels = ax.get_axis_labels(x_label=MathTex("x", color=BLACK), y_label=MathTex("y", color=BLACK))
        self.add(ax, labels)

        # Plot the points
        points = VGroup()
        for x, y in zip(x_data, y_data):
            point = Dot(ax.c2p(x, y), color=animat_blue)
            points.add(point)
        
        self.add(points)
        self.wait(1)

        # Draw a line between the min and max x values
        min_x, max_x = np.min(x_data), np.max(x_data)
        min_y, max_y = y_data[np.argmin(x_data)], y_data[np.argmax(x_data)]
        # line = ax.plot_line_graph([min_x, max_x], [min_y, max_y], line_color=YELLOW)
        slope = (max_y - min_y) / (max_x - min_x)
        intercept = max_y - slope * max_x
        line = ax.plot(lambda x: slope * x + intercept, color=animat_green) 
        line.set_z_index(1)
        points.set_z_index(2)

        self.play(Create(line))
        self.wait(1)

        # Add label 'f' at the end of the line
        line_as_vmobject = VMobject(color=animat_green).set_points(line.points)
        f_label = MathTex("f", font_size=36, color=animat_green).move_to(line_as_vmobject.get_end() + 0.5 * RIGHT)
        self.play(Write(f_label))

        # Compute the slope of the line
        slope = (max_y - min_y) / (max_x - min_x)

        # Compute the vertical distances between each point and the line
        y_on_line = slope * (x_data - min_x) + min_y  # Line values for each x_data
        distances = np.abs(y_data - y_on_line)

        # Find the index of the point with the largest distance
        largest_distance_index = np.argmax(distances)
        x0, y0 = x_data[largest_distance_index], y_data[largest_distance_index]
        y_on_line_for_x0 = y_on_line[largest_distance_index]

        # Mark the point with the largest distance
        point_of_interest = Dot(ax.c2p(x0, y0), color=animat_yellow)
        point_of_interest.set_z_index(3)  # Ensure the point is in front of the vertical line
        self.play(Indicate(point_of_interest, scale_factor=2))

        # Zoom into the point using the camera frame (with more zoom - 0.1)
        zoom_scale = 0.3
        zoomed_camera_frame = self.camera.frame
        zoomed_camera_frame.save_state()
        self.play(zoomed_camera_frame.animate.scale(zoom_scale).move_to(ax.c2p(x0, y0)))  # More zoom (0.1 for intense zoom)

        # Draw a vertical line from the point to the linear function
        # Set lower z_index for the line so it's behind the points
        vertical_line = DashedLine(
            start=ax.c2p(x0, y0),  # Start from the data point
            end=ax.c2p(x0, y_on_line_for_x0),  # End at the corresponding point on the linear function
            color=animat_red
        )
        vertical_line.set_z_index(1)  # Ensure the line is behind the points
        self.play(Create(vertical_line))

        # Mark the point on the line
        line_point = Dot(ax.c2p(x0, y_on_line_for_x0), color=BLACK)
        line_point.set_z_index(2)  # Ensure the white point is also in front of the vertical line
        self.play(FadeIn(line_point))

        # Label the points (making the math text even smaller)
        point_label = MathTex(f"({x0:.1f}, f({x0:.1f}))", font_size=10, color=BLACK).next_to(point_of_interest, 1/zoom_scale*UP)
        line_point_label = MathTex(f"({x0:.1f}, {y0:.1f})", font_size=10, color=BLACK).next_to(line_point, 1/zoom_scale*DOWN)
        
        point_label_data = MathTex(f"(x_{largest_distance_index}, f(x_{largest_distance_index}))", font_size=10, color=BLACK).next_to(point_of_interest, 1/zoom_scale*UP)
        line_point_label_data = MathTex(f"(x_{largest_distance_index}, y_{largest_distance_index})", font_size=10, color=BLACK).next_to(line_point, 1/zoom_scale*DOWN)
        
        point_label.set_z_index(3)
        line_point_label.set_z_index(3)
        self.play(Write(point_label), Write(line_point_label))

        self.wait(1)

        self.play(Transform(point_label, point_label_data),Transform(line_point_label, line_point_label_data))
        self.remove(labels)

        # Draw a square 
        # Set lower z_index for the line so it's behind the points
        dist = np.linalg.norm(np.array(ax.c2p(x0, y0)) - np.array(ax.c2p(x0, y_on_line_for_x0)))
        mid_point = (ax.c2p(x0, y0) + ax.c2p(x0, y_on_line_for_x0)) / 2
        square = Square(
            side_length=dist,
            color=animat_red
        )
        square.set_fill(animat_red, opacity=0.2)
        square.move_to(mid_point).shift(dist / 2 * RIGHT)
        square.set_z_index(1)  # Ensure the line is behind the points     
        
        # Label y - f(x) (smaller size)
        y_diff = MathTex(f"y_{largest_distance_index} - f(x_{largest_distance_index})", font_size=10, color=BLACK).move_to(square.get_center())
        y_diff.set_z_index(2)

        self.play(Write(y_diff))
        self.wait(2)
      
        # Square y - f(x)
        self.play(Create(square))   
        self.remove(vertical_line)

        squared_diff = MathTex(f"(y_{largest_distance_index} - f(x_{largest_distance_index}))^2", font_size=10, color=BLACK).move_to(square.get_center())
        squared_diff.set_z_index(2)

        self.play(Transform(y_diff, squared_diff))
        self.wait(5)

        # Restore the camera to its original state
        self.play(Restore(zoomed_camera_frame))
        self.wait(5)

For at sikre at alle afstande bidrager positivt til en samlet sum, så kvadrerer vi disse størrelser. Vi kigger så at sige på arealet af kvadraterne, som er udgjort af de lodrette afstande. Arealet af kvadratet, udgjort af det første datapunkt og punktet på linjen, er jo netop

$$

    (y_1 - f(x_1))(y_1 - f(x_1)) = (y_1 - f(x_1))^2.

$$

Fra {numref}`Kapitel %s<potenser>` ved vi, at et tal i anden potens altid er positivt (uanset om grundtallet er negativt eller positivt).

In [ ]:
from manim import *

In [1]:
%%manim -v WARNING -qm --format=mp4 AdjustingSlope

config.media_embed = True
config.media_width = "100%"

class AdjustingSlope(Scene):
    def construct(self):
        self.camera.background_color = WHITE
        
        # Set up the axes
        ax = Axes(
            x_range=[0, max(x_data)+2, 2],
            y_range=[0, max(y_data)+2, 2],
            axis_config={"include_numbers": True, "color": BLACK},
        )
        ax.x_axis.numbers.set_color(BLACK)
        ax.y_axis.numbers.set_color(BLACK)
        labels = ax.get_axis_labels(x_label=MathTex("x", color=BLACK), y_label=MathTex("y", color=BLACK))

        # Plot the points
        points = VGroup()
        for x, y in zip(x_data, y_data):
            point = Dot(ax.c2p(x, y), color=animat_blue)
            points.add(point)
        
        # Draw a line between the min and max x values
        min_x, max_x = np.min(x_data), np.max(x_data)
        min_y, max_y = y_data[np.argmin(x_data)], y_data[np.argmax(x_data)]

        slope = (max_y - min_y) / (max_x - min_x)
        intercept = max_y - slope * max_x
        line = ax.plot(lambda x: slope * x + intercept, color=animat_green) 
        line.set_z_index(1)
        points.set_z_index(2)

        # Compute the slope of the line
        slope = (max_y - min_y) / (max_x - min_x)

        # Compute the vertical distances between each point and the line
        y_on_line = slope * (x_data - min_x) + min_y  # Line values for each x_data
        distances = np.abs(y_data - y_on_line)

        # Find the index of the point with the largest distance
        largest_distance_index = np.argmax(distances)
        x0, y0 = x_data[largest_distance_index], y_data[largest_distance_index]
        y_on_line_for_x0 = y_on_line[largest_distance_index]

        # Mark the point with the largest distance
        point_of_interest = Dot(ax.c2p(x0, y0), color=animat_yellow)
        point_of_interest.set_z_index(3)

        # Mark the point on the line
        line_point = Dot(ax.c2p(x0, y_on_line_for_x0), color=BLACK)
        line_point.set_z_index(2)  # Ensure the white point is also in front of the vertical line

        # Label the points
        zoom_scale = 0.3
        point_label_data = MathTex(f"(x_{largest_distance_index}, f(x_{largest_distance_index}))", font_size=10, color=BLACK).next_to(point_of_interest, 1/zoom_scale*UP)
        line_point_label_data = MathTex(f"(x_{largest_distance_index}, y_{largest_distance_index})", font_size=10, color=BLACK).next_to(line_point, 1/zoom_scale*DOWN)

        # Function to get the slope and intercept of the rotated line
        def get_slope_intercept():
            rotated_slope = np.tan(np.arctan(slope) + angle_tracker.get_value() * DEGREES)
            mid_x = (min_x + max_x) / 2
            mid_y = (min_y + max_y) / 2
            rotated_intercept = mid_y - rotated_slope * mid_x
            return rotated_intercept, rotated_slope

        # Initialize animating rotation line
        line_as_vmobject = VMobject(color=animat_green).set_points(line.points)
        line_as_vmobject.set_z_index(1)
        line_as_vmobject_ref = line_as_vmobject.copy()

        angle_tracker = ValueTracker(0)
        rotation_center = ax.c2p((min_x + max_x) / 2, (min_y + max_y) / 2)

        line_as_vmobject.add_updater(
            lambda mob: mob.become(
                ax.plot(lambda x: get_slope_intercept()[1] * x + get_slope_intercept()[0], color=animat_green)
            )
        )

        f_label = MathTex("f", font_size=36, color=animat_green).move_to(line_as_vmobject.get_end() + 0.5 * RIGHT)

        f_label.add_updater(
            lambda x: x.move_to(
                line_as_vmobject.get_end() + 0.5 * RIGHT
            )
        )

        # Draw a square for the largest distance point
        dist = np.linalg.norm(np.array(ax.c2p(x0, y0)) - np.array(ax.c2p(x0, y_on_line_for_x0)))
        mid_point = (ax.c2p(x0, y0) + ax.c2p(x0, y_on_line_for_x0)) / 2
        square = Square(
            side_length=dist,
            color=animat_red,
            fill_color=animat_red,
            fill_opacity=0.2
        )
        square.move_to(mid_point).shift(dist / 2 * RIGHT)
        square.set_z_index(1)  # Ensure the line is behind the points     

        squared_diff = MathTex(f"(y_{largest_distance_index} - f(x_{largest_distance_index}))^2", font_size=10, color=BLACK).move_to(square.get_center())
        squared_diff.set_z_index(2)

        # Updater for the square with the largest distance point
        def update_square(square, x, y):
            current_intercept, current_slope = get_slope_intercept()
            y_on_line = current_slope * x + current_intercept
            new_dist = np.linalg.norm(np.array(ax.c2p(x, y)) - np.array(ax.c2p(x, y_on_line)))
            new_mid_point = (ax.c2p(x, y) + ax.c2p(x, y_on_line)) / 2
            square.become(Square(
                side_length=new_dist,
                color=animat_red,
                fill_color=animat_red,
                fill_opacity=0.2
            ).move_to(new_mid_point).shift(new_dist / 2 * RIGHT))

        square.add_updater(lambda mob: update_square(mob, x0, y0))

        self.add(ax, labels, points, line_as_vmobject, square, squared_diff, f_label)

        self.play(FadeOut(point_of_interest, line_point_label_data, point_label_data, line_point, squared_diff))

        # Draw the rest of the squares
        x_data_without_x0 = np.delete(x_data, largest_distance_index)
        y_data_without_x0 = np.delete(y_data, largest_distance_index)

        squares = VGroup()
        for x, y in zip(x_data_without_x0, y_data_without_x0):
            dist = np.linalg.norm(np.array(ax.c2p(x, y)) - np.array(ax.c2p(x, slope * (x - min_x) + min_y)))
            mid_point = (ax.c2p(x, y) + ax.c2p(x, slope * (x - min_x) + min_y)) / 2
            square = Square(
                side_length=dist,
                color=animat_red,
                fill_color=animat_red,
                fill_opacity=0.2
            ).move_to(mid_point).shift(dist / 2 * RIGHT)
            square.add_updater(lambda mob, x=x, y=y: update_square(mob, x, y))  # Add updaters to all squares
            squares.add(square)

        self.add(squares)
        self.play(Create(squares), run_time=2)

        # Rotate the line and adjust the squares dynamically
        self.play(angle_tracker.animate.set_value(10), run_time=2)
        self.wait(1)
        self.play(angle_tracker.animate.set_value(-20), run_time=2)
        self.wait(1)
        self.play(angle_tracker.animate.set_value(0), run_time=2)

        # Stop updaters after the animation
        line_as_vmobject.clear_updaters()
        squares.clear_updaters()
        self.wait(2)


UsageError: Cell magic `%%manim` not found.


Til sidst summer vi alle de kvadrerede afstande

$$

    \text{RSS} = (y_1 - f(x_1))^2 + (y_2 - f(x_2))^2 + \cdots + (y_n - f(x_n))^2 = \sum_{i=1}^n (y_i - f(x_i))^2,

$$ (eq:mindste-kvadraters-metode)

for at få ét samlet tal, der repræsenterer en samlet *fejl* for vores *regressionslinje*. Som du nok kan forstille dig; hvis datapunkterne ligger tæt på regressionslinjen, så vil de kvadrerede afstande også være små, og dermed vil summen af dem også være lille. Denne størrelse kaldes *Residual Sum of Squares* og forkortes $\text{RSS}$.

Simpel lineær regression med mindste kvadraters metode kan samtlige CAS-værktøjer lave for dig. Nogle steder hedder det bare at lave "lineær regression", mens andre steder kan det hedde at finde en "tendenslinje". 